In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import json
from flatten_json import flatten
from datetime import datetime
from collections import defaultdict


In [ ]:
## Convert all files to json ##

directory = ['2018','2019','2020','2021','2022','2023']

for year in directory:
    for filename in os.listdir(year):
        file_path = os.path.join(year, filename)
        print(file_path)
        
        if os.path.isfile(file_path) and '.' not in filename:
            new_file_path = f"{file_path}.json"
            os.rename(file_path, new_file_path)



In [32]:
 ## convert data into csv (includes only interested topics) ##

def process_file(file_path):
    """Process a single JSON file to extract required data."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Extract publication metadata
        coredata = data.get('abstracts-retrieval-response', {}).get('coredata', {})
        cover_date = data.get('abstracts-retrieval-response', {}).get('coredata', {}).get('prism:coverDate', None)
        publication_name = coredata.get('prism:publicationName', 'Unknown')
        title = coredata.get('dc:title', 'Unknown')
        author = coredata.get('dc:creator', {}).get('author', [{}])[0].get('ce:indexed-name', 'Unknown')
        publisher = coredata.get('dc:publisher', 'Unknown')

        # Extract subject areas
        subject_areas = data.get('abstracts-retrieval-response', {}).get('subject-areas', {}).get('subject-area', [])
        subject_abbrevs = [area.get('@abbrev', 'Unknown') for area in subject_areas]


        # Return as list of records
        return [
            {   "Cover-Date": cover_date,
                "Publication Name": publication_name,
                "Title": title,
                "Author": author,
                "Publisher": publisher,
                "Subject Area":list(set(subject_abbrevs))
            }
        ]
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return []

def process_all_files_to_dataframe(base_dir):
    """Process all files and return a DataFrame with the required columns."""
    all_records = []

    for year_folder in os.listdir(base_dir):
        year_path = os.path.join(base_dir, year_folder)
        if os.path.isdir(year_path):
            for file_name in os.listdir(year_path):
                file_path = os.path.join(year_path, file_name)
                if file_path.endswith('.json'):
                    # Process each JSON file and collect records
                    records = process_file(file_path)
                    all_records.extend(records)

    # Convert to DataFrame
    return pd.DataFrame(all_records)

# Base directory where project files are stored
base_dir = "/Users/im/Documents/Data Sci/Project/Project/chulaDatabase"

# Process all files and create DataFrame
df = process_all_files_to_dataframe(base_dir)


# Assuming 'df' is your DataFrame
df['Subject Area'] = df['Subject Area'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# drop null and duplicate values

df = df.dropna()
df = df.drop_duplicates()
df.reset_index(drop=True)

# Save to a CSV for analysis
output_file = "full_Chula_data.csv"
df.to_csv(output_file, index=False)

# Display first few rows of the DataFrame
df


,Cover-Date,Publication Name,Title,Author,Publisher,Subject Area
0,2022-05-01,Animal Bioscience,Microencapsulated basil oil (Ocimum basilicum ...,Thuekeaw S.,Asian-Australasian Association of Animal Produ...,"AGRI, VETE, BIOC"
1,2022-04-01,Journal of Neurosurgical Anesthesiology,Perceptions Regarding the SARS-CoV-2 Pandemic'...,Lele A.V.,Lippincott Williams and Wilkins,MEDI
2,2022-01-01,International Journal of Neuroscience,Construction of a short version of the Montrea...,Hemrungrojn S.,Taylor and Francis Ltd.,NEUR
3,2022-02-14,Journal of Applied Physics,The effect of strain and pressure on the elect...,Johansson E.,American Institute of Physics Inc.,PHYS
4,2022-01-01,Journal of Exercise Physiology Online,Dynamic Cardiopulmonary and Metabolic Function...,Masodsai K.,American Society of Exercise Physiologists,MEDI
...,...,...,...,...,...,...
20211,2018-01-01,Frontiers in Artificial Intelligence and Appli...,Estimating actual evapotranspiration from NDVI...,Jermthaisong P.,IOS PressNieuwe Hemweg 6BAmsterdam1013 BG,COMP
20212,2018-12-01,Scientific Reports,Genome-wide association study identified new s...,Sawai H.,"Nature Publishing GroupHoundmillsBasingstoke, ...",MULT
20213,2018-01-01,Journal of the Medical Association of Thailand,Effects of transcranial direct current stimula...,Utarapichat S.,Medical Association of Thailandmath@loxinfo.co.th,MEDI
20214,2018-09-01,Thai Journal of Veterinary Medicine,Prevalence and risk factors for canine cogniti...,Benjanirut C.,Chulalongkorn University Printing House39 Henr...,VETE


In [33]:
## Number of researchs based on subject areas in each year ##

df['Year'] = pd.to_datetime(df['Cover-Date']).dt.year

# Split 'Subject Area' into individual entries
df = df.assign(Subject_Area=df['Subject Area'].str.split(', '))

# Explode the 'Subject Area' list into separate rows
df = df.explode('Subject_Area')

# Group by 'Year' and 'Subject_Area' and count occurrences
grouped = df.groupby(['Year', 'Subject_Area']).size().reset_index(name='Count')

# Pivot the data to reshape it
pivot_table = grouped.pivot(index='Year', columns='Subject_Area', values='Count').fillna(0).astype(int)
pivot_table.columns.name = None

pivot_table.to_csv('areasCount_ChulaData.csv', index=True)


pivot_table


,AGRI,ARTS,BIOC,BUSI,CENG,CHEM,COMP,DECI,DENT,EART,...,MATH,MEDI,MULT,NEUR,NURS,PHAR,PHYS,PSYC,SOCI,VETE
Year,,,,,,,,,,,,,,,,,,,,,
2018,254,33,343,71,199,302,236,33,57,103,...,101,749,84,84,38,153,388,24,157,106
2019,231,79,370,100,214,314,358,87,52,108,...,124,842,101,84,33,151,384,21,203,84
2020,321,86,381,124,242,330,294,86,50,148,...,127,907,147,73,45,180,355,34,222,91
2021,354,110,461,100,313,440,286,31,82,142,...,124,1107,256,86,67,196,373,37,320,109
2022,431,108,515,126,335,447,402,80,105,109,...,154,1230,309,97,78,237,338,64,366,141
2023,310,86,366,81,252,328,250,36,101,103,...,110,804,191,81,60,139,234,46,247,60
